### Ice Shapes
S4.4 of Gerakines & Hudson 2020 (https://iopscience.iop.org/article/10.3847/1538-4357/abad39) gives "IR absorption coefficient α(ν̃) (in cm−1)":
$$\alpha(\tilde{\nu}) = 4 \pi \tilde{\nu} k(\tilde{\nu})$$
and the band strength A (in cm molecule−1):
$$A = \frac{1}{\rho_N} \int_{band} \alpha(\tilde{\nu})d \tilde{\nu}$$

where $\rho_N$ is the number density of the absorbers (in molecules cm$^{-3}$) 

$\alpha(\tilde{\nu})$ is the IR absorption coefficient in cm$^{-1}$.

$A$ is the band strength in cm/molecule

Hudgins 1993 (https://ui.adsabs.harvard.edu/abs/1993ApJS...86..713H/abstract) eqn 8 is:
$$A = \frac{1}{N}\int \tau d\nu$$
where $N$ is the number density of molecules (molecules per area).

Some algebra to get $\tau$, the observable, from the data:

$$A N = \int \tau_\nu d\nu$$

$$\frac{d}{d\nu} A N = \frac{d}{d\nu}\int \tau_\nu d\nu$$

$$N \frac{d}{d\nu} A = \tau_\nu $$

$$N \frac{\alpha_\nu}{\rho_N} = \tau_\nu $$

$$\tau_\nu = N \frac{\alpha_\nu}{\rho_N} = N \frac{4 \pi \tilde{\nu} k}{\rho_N}$$

### Compare to Gerakines

Verifying that the integral of opacity equals A

A in Table 3 is about $1\times10^{-17}$ cm/molecule

In [5]:
from icemodels.core import retrieve_gerakines_co, absorbed_spectrum
gerakines_tbl = retrieve_gerakines_co()

In [27]:
import numpy as np
from astropy import units as u
xarr = np.linspace(4.6, 4.75, 1000)*u.um
ice_column = 1 * u.cm**-2
opacity = absorbed_spectrum(ice_column=ice_column, ice_model_table=gerakines_tbl, return_tau=True, xarr=xarr)

<Quantity [8.35242307e-21, 8.28931703e-21, 8.22420604e-21, 8.15146534e-21,
           8.07902293e-21, 8.00700286e-21, 7.93846421e-21, 7.87153472e-21,
           7.81897975e-21, 7.77046002e-21, 7.73071332e-21, 7.69855289e-21,
           7.67113613e-21, 7.64692031e-21, 7.62334126e-21, 7.60100099e-21,
           7.57739727e-21, 7.55260493e-21, 7.52988952e-21, 7.50804551e-21,
           7.48929178e-21, 7.47520456e-21, 7.46773716e-21, 7.46512441e-21,
           7.46456604e-21, 7.47233527e-21, 7.48004940e-21, 7.48764148e-21,
           7.49515176e-21, 7.50260921e-21, 7.50502712e-21, 7.50633633e-21,
           7.50535436e-21, 7.50003683e-21, 7.49043022e-21, 7.47259888e-21,
           7.45069430e-21, 7.41015081e-21, 7.36042980e-21, 7.29666534e-21,
           7.21337439e-21, 7.12141159e-21, 7.01187050e-21, 6.89960917e-21,
           6.78102285e-21, 6.66839570e-21, 6.56003871e-21, 6.46585118e-21,
           6.37777603e-21, 6.30927348e-21, 6.24366403e-21, 6.18129991e-21,
           6.12022565e-21

In [28]:
dnu = np.abs(xarr.to(u.cm**-1, u.spectral()).diff())
(opacity[:-1]*dnu / ice_column).sum().to(u.cm)

<Quantity 1.42014076e-17 cm>